In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
# get environment variable: OPENAI_API_KEY
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import os
import openai
import tiktoken
import time
import numpy as np
import pandas as pd

In [26]:
openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# find tokens for 'positive' and 'negative'
enc = tiktoken.encoding_for_model("gpt-4")
enc.encode("positive"), enc.encode("negative") 

([31587], [43324])

In [6]:
# download dataset from hf datasets https://github.com/huggingface/datasets
# !pip install datasets 
from datasets import load_dataset

dataset = load_dataset("sst2")

Found cached dataset sst2 (/home/survival/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
val_data = dataset["validation"]

In [8]:
# sentimentid2sentimentlabel = {"0": "negative", "1": "positive"}
sents, labels = [], []
for idx in range(len(val_data)):
    sents.append(val_data[idx]["sentence"])
    label = val_data[idx]["label"]
    if label == 0:
        labels.append('negative')
    else:
        labels.append('positive')

In [9]:
# Not needed if you have already access to gpt-4. 
OPENROUTER_REFERRER = "https://github.com/alonsosilvaallende/chatplotlib-openrouter"

In [10]:
def my_classifier(text: str) -> str:
    return openai.ChatCompletion.create(
    model='openai/gpt-4',
    headers={"HTTP-Referer": OPENROUTER_REFERRER}, # Not needed if you already have access to gpt-4
    messages=[
        {
            'role': 'system',
            'content': 'You are a helpful sentiment analyzer assistant of movie reviews'
        },
        {
        'role':'user',
        'content':f'Given the following text, what is the sentiment conveyed? Is it positive or negative?: {text}'
    }],
    logit_bias={
        '31587': 100,    # 31587 is the token for `positive`
        '43324': 100     # 43324 is the token for `negative`
    },
    max_tokens=1,
    temperature=0
).choices[0].message.content

In [11]:
preds = []
for i, sent in enumerate(sents):
    try:
        preds.append(my_classifier(sent))
    except:
        preds.append(np.nan)
        time.sleep(3)

In [12]:
import pandas as pd

missing = [i for i, pred in enumerate(preds) if pd.isna(pred)]
missing

[505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 576,
 577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 588,
 589,
 590,
 591,
 592,
 593,
 594,
 595,
 596,
 597,
 598,
 599,
 600,
 601,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 609,
 610,
 611,
 612,
 613,
 614,
 615,
 616,
 617,
 618,
 619,
 620,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 648,
 649,
 650,
 651,
 652,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 667,
 668,
 669,
 670,
 671

In [29]:
missing_preds = []
for idx in missing:
    try:
        missing_preds.append(my_classifier(sents[idx]))
    except:
        missing_preds.append(np.nan)
        time.sleep(3)

In [30]:
for i, idx in enumerate(missing):
    preds[idx] = missing_preds[i]

In [31]:
missing = [i for i, pred in enumerate(preds) if pd.isna(pred)]
missing

[]

In [32]:
np.mean([labels[i] == preds[i] for i in range(len(preds))])

0.9598623853211009

In [33]:
results = pd.DataFrame()
results["sentences"] = sents
results["labels"] = labels
results["predictions"] = preds

In [34]:
results.to_csv("./outputs/SST2_dataset_results_gpt-4.csv", index=False)